In [1]:
# tf tools
import tensorflow as tf

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)
# cifar10 data - 32x32
from tensorflow.keras.datasets import cifar10

# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model

# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD

#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import zipfile

2023-05-21 11:50:22.442075: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Unzipping file

In [4]:
    folder_path = os.path.join("..", "data", "Test_Alphabet") # Path to the data if unzipped already
    if not os.path.exists(folder_path): # Checking to see if folder is unzipped
        print("Unzipping file")
        path_to_zip = os.path.join("..","data","archive.zip") # Defining the path to the zip file
        zip_destination = os.path.join("..", "data") # defining the output destination

        with zipfile.ZipFile(path_to_zip,"r") as zip_ref: # using the package from zipfile, to un zip the zip file
            zip_ref.extractall(zip_destination)
        
    print("The files are unzipped")

The files are unzipped


NameError: name 'zip_destination' is not defined

## Deleting blank folder 

OSError: [Errno 39] Directory not empty: '../data/Test_Alphabet/Blank'

## Image datagenerator 

In [3]:

    print("Creating Image data generator")
    # ImageDataGenerator from tensorflow 
    datagen = ImageDataGenerator(horizontal_flip=True,
                                validation_split = 0.2, # Flip it horizontally around the access randomly 
                                 # Rotate the image randomly 20 degress around the access
                                rescale = 1/255 # rescale the pixel values to between 0-1
                                
    )


Creating Image data generator


## Training images

## Flow from directory

In [4]:
directory = os.path.join("..","data","Train_Alphabet")

In [5]:
training_tensorflow = datagen.flow_from_directory(
    directory,
    target_size=(90, 90),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    #save_to_dir=None,
    #save_prefix='',
    #save_format='png',
    #follow_links=False,
    subset="training",
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 18720 images belonging to 26 classes.


## Validation

In [6]:
validation_tensorflow = datagen.flow_from_directory(
    directory,
    target_size=(90, 90),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    #save_to_dir=None,
    #save_prefix='',
    #save_format='png',
    #follow_links=False,
    subset="validation",
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 4680 images belonging to 26 classes.


## Test images

In [7]:
    test_datagen = ImageDataGenerator(
                                    rescale = 1./255. # Datagenerator for test, it only has to rescale the images 
    )

In [8]:
directory_test = os.path.join("..","data","Test_Alphabet")

In [9]:
test_tensorflow_ = test_datagen.flow_from_directory(
    directory_test,
    target_size=(90, 90),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    seed=None,
    #save_to_dir=None,
    #save_prefix='',
    #save_format='png',
    #follow_links=False,
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 2600 images belonging to 26 classes.


## Loading model

In [10]:

    print("Loading model: ")  
    # load model without classifier layers
    model = VGG16(include_top=False, # Exclude classifier layers
                pooling='avg',
                input_shape=(90, 90, 3)) # Input shape of the images. 224 pixels by 224. 3 color channels

    # Keep pretrained layers, and don't modify them
    for layer in model.layers:
        layer.trainable = False
        
    # Add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    bn = BatchNormalization()(flat1) # Added batnormalization from tensorflow. Take the previouslayer, normalise the values, and than pass them on
    class1 = Dense(256, 
                activation='relu')(bn) # Added new classification layer 
    class2 = Dense(128, 
                activation='relu')(class1) # Added new classification layer with 15 outputs. 15 labels in total
    output = Dense(26, # 15 labels
                activation='softmax')(class2)

    # define new model
    model = Model(inputs=model.inputs, 
                outputs=output)

    # compile
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01, # Start learning rate at 0.01
        decay_steps=10000, # Every 10 000 steps start decaying 
        decay_rate=0.9) # DEcay by 0.9 to the start learning rate
    sgd = SGD(learning_rate=lr_schedule)

    model.compile(optimizer=sgd,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    # summarize
    print(model.summary())
    

Loading model: 
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90, 90, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 90, 90, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 90, 90, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 45, 45, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 45, 45, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 45, 45, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 22, 22, 1

In [11]:
    H = model.fit( # fitting the model to 
        training_tensorflow, # training data from tensorflow dataframe 
        steps_per_epoch = len(training_tensorflow), # Take as many steps as the length of the dataframe 
        validation_data = validation_tensorflow, # Validation data from tensorflow dataframe
        validation_steps = len(validation_tensorflow), # Validation steps as length of validation data 
        epochs = 5)

Epoch 1/5


2023-05-20 15:30:43.173703: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


585/585 [==============================] - ETA: 0s - loss: 2.9869 - accuracy: 0.1561

2023-05-20 15:59:22.470276: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


585/585 [==============================] - 2200s 4s/step - loss: 2.9869 - accuracy: 0.1561 - val_loss: 2.6286 - val_accuracy: 0.2733
Epoch 2/5
585/585 [==============================] - 1394s 2s/step - loss: 2.3219 - accuracy: 0.3315 - val_loss: 2.1597 - val_accuracy: 0.3682
Epoch 3/5
585/585 [==============================] - 1246s 2s/step - loss: 1.9837 - accuracy: 0.4188 - val_loss: 1.9360 - val_accuracy: 0.4256
Epoch 4/5
585/585 [==============================] - 235s 401ms/step - loss: 1.7982 - accuracy: 0.4644 - val_loss: 1.8187 - val_accuracy: 0.4632
Epoch 5/5
585/585 [==============================] - 233s 399ms/step - loss: 1.6783 - accuracy: 0.4987 - val_loss: 1.7511 - val_accuracy: 0.4808


## Predictions 

In [12]:
# Code taken from source in readme file
# Testing the model
print("Testing model with test images")
# Predict the label of the test_images
pred = model.predict(test_tensorflow_) # Using test data on the model
pred = np.argmax(pred,axis=1)

# Map the label 
labels = (training_tensorflow.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]


Testing model with test images


2023-05-20 17:00:06.730652: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


82/82 [==============================] - 26s 314ms/step


In [14]:
labels = dict((v,k) for k,v in labels.items())

In [13]:
true_labels = test_tensorflow_.classes

# Convert labels to numerical values
label_map = {v: k for k, v in training_tensorflow.class_indices.items()}
true_labels = np.array([label_map[label] for label in true_labels])

# Print the classification report
print(classification_report(true_labels, pred))


              precision    recall  f1-score   support

           A       0.55      0.59      0.57       100
           B       0.40      0.65      0.50       100
           C       0.75      0.67      0.71       100
           D       0.52      0.44      0.48       100
           E       0.36      0.35      0.36       100
           F       0.40      0.44      0.42       100
           G       0.64      0.61      0.63       100
           H       0.69      0.70      0.70       100
           I       0.54      0.61      0.57       100
           J       0.44      0.48      0.46       100
           K       0.44      0.24      0.31       100
           L       0.40      0.62      0.49       100
           M       0.36      0.36      0.36       100
           N       0.38      0.30      0.34       100
           O       0.64      0.71      0.67       100
           P       0.51      0.46      0.48       100
           Q       0.34      0.45      0.38       100
           R       0.47    

## Saving model

In [15]:

folder_path = os.path.join("..","models", "fake_model.keras") # Defining out path
tf.keras.models.save_model( # Using Tensor Flows function for saving models.
model, folder_path, overwrite=False, save_format=None 
) # Model name, folder, Overwrite existing saves, save format = none 

## Trying on real test data

In [16]:

# Set the path to the directory containing your data
data_dir = os.path.join("..","data","asl_alphabet_train","asl_alphabet_train")



# Initialize lists to store image paths and labels
image_paths = []
labels = []
images = []

# Iterate through each letter folder in the data directory
for label_folder in os.listdir(data_dir):
    letter_folder = os.path.join(data_dir, label_folder)

    # Iterate through each image file in the letter folder
    for filename in os.listdir(letter_folder):
        image_path = os.path.join(letter_folder, filename)
        images.append(image_path)
        # Append the image path and label to the respective lists
        image_paths.append(image_path)
        labels.append(label_folder)

# Create a DataFrame from the image_paths and labels lists
train_df = pd.DataFrame({'image_path': image_paths, 'label': labels})

In [17]:
test_images_dataframe = pd.DataFrame(columns = ["image_path", "label"]) # Creating empty dataframe

labels_grouped = train_df.groupby("label") # Grouping the dataframe by label

for label, group in labels_grouped: # takes the first 100 of each label and concatinates them into the new dataframe
    test_images = group.head(100)
    test_images_dataframe = pd.concat([test_images_dataframe, test_images]) 

train_df = train_df.drop(test_images_dataframe.index) # Removing the images with the same index

In [20]:
test_tensorflow_ = test_datagen.flow_from_dataframe(
    test_images_dataframe,
    directory=None,
    x_col='image_path',
    target_size=(90, 90),
    color_mode='rgb',
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=None
)

#["A","B","C","D","E","F","G","H","J","K","L","M","N","P","Q","R","S","T","U","V","W","X","Y","Z"],

Found 2600 validated image filenames.


In [21]:
# Code taken from source in readme file
# Testing the model
print("Testing model with test images")
# Predict the label of the test_images
pred = model.predict(test_tensorflow_) # Using test data on the model
pred = np.argmax(pred,axis=1)

# Map the label 
labels = (training_tensorflow.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]


Testing model with test images


2023-05-20 17:07:53.664855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


82/82 [==============================] - 132s 2s/step


In [22]:
true_labels = test_images_dataframe["label"].values
print(classification_report(true_labels, pred))

              precision    recall  f1-score   support

           A       0.04      0.12      0.06       100
           B       0.53      0.18      0.27       100
           C       0.08      0.19      0.12       100
           D       0.04      0.02      0.03       100
           E       0.14      0.21      0.17       100
           F       0.11      0.17      0.13       100
           G       0.10      0.06      0.08       100
           H       0.16      0.26      0.20       100
           I       0.27      0.07      0.11       100
           J       0.02      0.01      0.01       100
           K       0.32      0.13      0.18       100
           L       0.28      0.16      0.20       100
           M       0.13      0.02      0.03       100
           N       0.33      0.02      0.04       100
           O       0.66      0.31      0.42       100
           P       0.02      0.05      0.03       100
           Q       0.12      0.08      0.10       100
           R       0.09    